# 🎓 FrugalGPT: Performance and Cost Tradeoffs

This notebook illustrates the FrugalGPT framework for _building LLM Applications with budget constraints._

In particular, we will focus on evaluating the performance and cost tradeoffs enabled by FrugalGPT.

NB: You are highly suggested to use accelerated hardware (GPU/TPU) to run this notebook.

## Installation
Let us start by installing FrugalGPT (if you haven't yet!).

In [ ]:
# set up the environment
%%capture
! git clone https://github.com/stanford-futuredata/FrugalGPT
! pip install git+https://github.com/stanford-futuredata/FrugalGPT
!mkdir -p FrugalGPT/strategy
! wget  https://github.com/lchen001/DataHolder/releases/download/v0.0.3/COQA_Model20241128.zip
! unzip COQA_Model20241128.zip -d FrugalGPT/strategy/COQA_Model202411
! rm COQA_Model20241128.zip
! wget -P FrugalGPT/db/ https://github.com/lchen001/DataHolder/releases/download/v0.0.3/COQA.sqlite
!mkdir -p FrugalGPT/data/COQA
!wget -P FrugalGPT/data/COQA https://github.com/stanford-futuredata/FrugalGPT/releases/download/0.0.1/COQA_train.csv
!wget -P FrugalGPT/data/COQA https://github.com/stanford-futuredata/FrugalGPT/releases/download/0.0.1/COQA_test.csv


In [ ]:
%cd FrugalGPT

/content/FrugalGPT


In [ ]:
%load_ext autoreload
%autoreload 2
import sys, json, copy
import logging
logging.disable(logging.CRITICAL)
sys.path.append("src/")

## Setup
Next, let us set up the environment and API keys. You do _not_ need API keys to run the notebook! They are only needed if you want to use FrugalGPT for your own queries.
#### NB: _For your own queries, not all API keys are needed, too. If you only want to leverage LLMs from, e.g., OpenAI and AI21, setting up API keys for them is sufficient._

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_KEY'
os.environ['AI21_STUDIO_API_KEY'] = 'YOUR_AI21_KEY'
os.environ['COHERE_STUDIO_API_KEY'] = 'YOUR_COHERE_KEY'
os.environ['TEXTSYNTH_API_SECRET_KEY'] = 'YOUR_TEXTSYNTH_KEY'
os.environ['ANTHROPIC_API_KEY'] = 'YOUR_ANTHROPIC_KEY'
os.environ['TOGETHER_API_KEY'] = 'YOUR_TOGETHER_KEY'
os.environ['GEMINI_API_KEY'] = 'YOUR_GEMINI_KEY'

from IPython.display import display
import FrugalGPT
supported_LLM = FrugalGPT.getservicename()
print("supported LLMs:",supported_LLM)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

supported LLMs: ['textsynth/gptneox_20B', 'textsynth/fairseq_gpt_13B', 'textsynth/gptj_6B', 'openai/text-davinci-002', 'openai/text-davinci-003', 'openai/text-curie-001', 'openai/text-babbage-001', 'openai/text-ada-001', 'openaichat/gpt-4o-mini', 'openaichat/gpt-4o-mini-2024-07-18', 'openaichat/gpt-4o', 'openaichat/gpt-4o-2024-05-13', 'openaichat/gpt-4-turbo', 'openaichat/gpt-4o-2024-08-06', 'openaichat/gpt-3.5-turbo', 'openaichat/gpt-4', 'ai21/jamba-1.5-mini', 'ai21/jamba-1.5-large', 'ai21/j1-jumbo', 'ai21/j1-grande', 'ai21/j1-large', 'ai21/j2-ultra', 'ai21/j2-mid', 'ai21/j2-light', 'cohere/command', 'cohere/base', 'cohere/xlarge', 'cohere/medium', 'togetherai/Qwen/Qwen2-72B-Instruct', 'togetherai/mistralai/Mistral-7B-Instruct-v0.1', 'togetherai/google/gemma-2b-it', 'togetherai/google/gemma-2-9b-it', 'togetherai/google/gemma-2-27b-it', 'togetherai/meta-llama/Meta-Llama-3-8B-Instruct-Lite', 'togetherai/Qwen/Qwen1.5-110B-Chat', 'togetherai/mistralai/Mistral-7B-Instruct-v0.3', 'togethera

## Generating the tradeoffs involves three major steps: (i) prepare the dataset, (ii) train the FrugalGPT strategy, and (iii) evaluate and save the performance.

## Step 1: Prepare the dataset

In [ ]:
import pandas as pd
def list_to_dataframe(data_list):
    # The first sublist is the header
    headers = data_list[0]
    # The rest are the data rows
    data = data_list[1:]
    # Create the dataframe
    df = pd.DataFrame(data, columns=headers)
    return df

def convert_and_merge_dataframes(train_df, test_df):
    def extract_last_query_part(query):
        # Split the query by '\n\n' and take the last part
        return query.split('\n\n')[-1]

    def create_converted_df(df, start_query_id=1):
        # Extract the new 'query' and keep 'ref_answer' the same
        df['new_query'] = df['query'].apply(extract_last_query_part)

        # Group by 'new_query' and 'ref_answer' to merge identical queries
        merged_df = df.groupby(['new_query', 'ref_answer'], as_index=False).first()

        # Create a new dataframe with the three columns
        converted_df = pd.DataFrame({
            'query': merged_df['new_query'],
            'ref_answer': merged_df['ref_answer'],
            'query_id': range(start_query_id, start_query_id + len(merged_df))
        })

        return converted_df

    # Convert and merge the train dataframe
    converted_train_df = create_converted_df(train_df)

    # Find the last query_id from the training data
    last_train_query_id = converted_train_df['query_id'].max()

    # Convert and merge the test dataframe, starting query_id from the last training id + 1
    converted_test_df = create_converted_df(test_df, start_query_id=last_train_query_id + 1)

    return converted_train_df, converted_test_df


In [ ]:
train_df = pd.read_csv("data/COQA/COQA_train.csv")
train_df = train_df[train_df['model name'] == 'textsynth/gptneox_20B']

test_df = pd.read_csv("data/COQA/COQA_test.csv")
test_df = test_df[test_df['model name'] == 'textsynth/gptneox_20B']

In [ ]:
import pandas as pd
import os
from sklearn.utils import shuffle

# Combine the datasets
combined_df = pd.concat([train_df, test_df], ignore_index=True)

# Shuffle the combined dataframe
shuffled_df = shuffle(combined_df, random_state=42)

# Calculate the original proportions
train_size = len(train_df)
test_size = len(test_df)

# Split the dataframe back into train and test
new_train_df = shuffled_df.iloc[:train_size]
new_test_df = shuffled_df.iloc[train_size:]

# Create a new folder to save the files
output_folder = 'data/COQA_new/'
os.makedirs(output_folder, exist_ok=True)

# Save the new train and test CSVs
new_train_df.to_csv(os.path.join(output_folder, 'COQA_train.csv'), index=False)
new_test_df.to_csv(os.path.join(output_folder, 'COQA_test.csv'), index=False)

print(f"Reshuffled train and test CSVs saved in '{output_folder}'")


Reshuffled train and test CSVs saved in 'data/COQA_new/'


In [ ]:
train_raw = FrugalGPT.loadcsvdata("data/COQA_new/COQA_train.csv")
test_raw = FrugalGPT.loadcsvdata("data/COQA_new/COQA_test.csv")
train_df = list_to_dataframe(train_raw)
test_df = list_to_dataframe(test_raw)
converted_train, converted_test = convert_and_merge_dataframes(train_df, test_df)
columns_to_save = ['query', 'ref_answer', 'query_id']
converted_train[columns_to_save].to_csv("data/COQA_new/train.csv", index=False, header=False)
converted_test[columns_to_save].to_csv("data/COQA_new/test.csv",index=False, header=False)


## Step 2: Train the FrugalGPT strategy for different budgets

Let us first evaluate individual models.

In [ ]:
import pandas as pd

def generate_dataframe(service_names, train_data, test_data, genparams,db_path="db/SCIQ.sqlite",
                       max_workers=2):
    # Initialize an empty list to store the rows for the DataFrame
    data = []
    MyLLMforAll = FrugalGPT.LLMforAll(
                     db_path=db_path,
                     max_workers=max_workers,

)
    # Dictionary to keep track of markers for each provider
    provider_marker = {}

    # Iterate through the service names
    for name in service_names:
        # Extract provider and method
        provider = name.split('/')[0]
        method = name.split('/')[-1]

        # If the provider is seen for the first time, initialize its marker
        if provider not in provider_marker:
            provider_marker[provider] = 1
        else:
            provider_marker[provider] += 1
        # Get the completion batch for train and test data
        r1_train = MyLLMforAll.get_completion_batch(queries=train_data, genparams=genparams, service_name=name)
        r2_train = FrugalGPT.compute_score(r1_train)
        r1_test = MyLLMforAll.get_completion_batch(queries=test_data, genparams=genparams, service_name=name)
        r2_test = FrugalGPT.compute_score(r1_test)

        # Extract accuracy and cost
        train_acc = r2_train['em']
        train_cost = r2_train['cost']
        test_acc = r2_test['em']
        test_cost = r2_test['cost']

        # Create a row with the schema
        row = {
            "Test_acc": test_acc,
            "Test_cost": test_cost,
            "Test_size": len(test_data),
            "Train_acc": train_acc,
            "Train_cost": train_cost,
            "Train_size": len(train_data),
            "Budget": 10,
            "Method": method,
            "Provider": provider,
            "Marker": provider_marker[provider],
        }

        # Append the row to the data list
        data.append(row)

    # Create the DataFrame from the data list
    df = pd.DataFrame(data)

    return df

In [ ]:
dataname = "COQA"
service_names = [
    'openaichat/gpt-4o-2024-05-13',
    #'openaichat/gpt-4o-2024-08-06',
    'openaichat/gpt-4o-mini',
    #'openaichat/gpt-4o',
    'openaichat/gpt-4-turbo',

    'google/gemini-1.5-flash',
    'google/gemini-1.5-pro',
    'google/gemini-1.5-flash-8b',

    #'togetherai/mistralai/Mistral-7B-Instruct-v0.1',
    #'togetherai/Qwen/Qwen2-72B-Instruct',
    #'togetherai/meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo',
    'togetherai/meta-llama/Meta-Llama-3-70B-Instruct-Turbo',
    'togetherai/google/gemma-2-9b-it',

    'anthropic/claude-3-5-sonnet-20240620',
    #'anthropic/claude-3-haiku-20240307',

    #'ai21/jamba-1.5-mini',
    'ai21/jamba-1.5-large',
                 ]
genparams=FrugalGPT.GenerationParameter(max_tokens=50, temperature=0.1, stop=['\n'])

test_data = FrugalGPT.loadcsvdata(f"data/{dataname}/test.csv")
prefix = open(f'config/prompt/{dataname}/prefix_e2.txt').read()
test_data = FrugalGPT.formatdata(test_data,prefix)

train_data = FrugalGPT.loadcsvdata(f"data/{dataname}/train.csv")
prefix = open(f'config/prompt/{dataname}/prefix_e2.txt').read()
train_data = FrugalGPT.formatdata(train_data,prefix)

sample_size = 10000
individualmodel_df = generate_dataframe(service_names,
                                        train_data[0:sample_size], test_data[0:sample_size],
                                        genparams,
                                        db_path=f"db/{dataname}.sqlite",
                                        max_workers=4)
display(individualmodel_df)
individualmodel_df.to_csv(f"summary_{dataname}_e2_2024.csv")


3991it [00:05, 687.11it/s]
3991it [00:05, 705.54it/s]
3991it [00:05, 688.86it/s]
3991it [00:05, 706.94it/s]
3991it [00:06, 652.52it/s]
3991it [00:05, 685.37it/s]
3991it [00:06, 650.57it/s]
3991it [00:05, 670.07it/s]
3991it [00:06, 644.93it/s]
3991it [00:05, 670.36it/s]
3991it [00:06, 664.08it/s]
3991it [00:05, 665.64it/s]
3991it [00:05, 697.44it/s]
3991it [00:05, 716.66it/s]
3991it [00:05, 698.34it/s]
3991it [00:05, 719.42it/s]
3991it [00:06, 653.78it/s]
3991it [00:06, 648.26it/s]
3991it [00:06, 648.42it/s]
3991it [00:06, 641.99it/s]


,Test_acc,Test_cost,Test_size,Train_acc,Train_cost,Train_size,Budget,Method,Provider,Marker
0,0.162866,0.004758,3991,0.175395,0.004762,3991,10,gpt-4o-2024-05-13,openaichat,1
1,0.116262,0.000145,3991,0.131546,0.000145,3991,10,gpt-4o-mini,openaichat,2
2,0.277374,0.014052,3991,0.295164,0.014052,3991,10,gpt-4-turbo,openaichat,3
3,0.338762,0.000073,3991,0.348284,0.000073,3991,10,gemini-1.5-flash,google,1
4,0.275370,0.001235,3991,0.288649,0.001234,3991,10,gemini-1.5-pro,google,2
5,0.231020,0.000037,3991,0.249562,0.000037,3991,10,gemini-1.5-flash-8b,google,3
6,0.253821,0.000815,3991,0.263343,0.000816,3991,10,Meta-Llama-3-70B-Instruct-Turbo,togetherai,1
7,0.206715,0.000280,3991,0.212729,0.000280,3991,10,gemma-2-9b-it,togetherai,2
8,0.283638,0.003287,3991,0.290403,0.003287,3991,10,claude-3-5-sonnet-20240620,anthropic,1
9,0.154347,0.002022,3991,0.161363,0.002021,3991,10,jamba-1.5-large,ai21,1


In [ ]:
from google.colab import files
#files.download(f"db/{dataname}.sqlite")

Now let us train FrugalGPT on this dataset.

In [ ]:
import numpy
from tqdm import tqdm
def compute_tradeoffs(
    train_data,
                      budget_list,
                      name = "test",

                      service_names = ['openaichat/gpt-4o-mini',
                                       'openaichat/gpt-4o',
                                      'openaichat/gpt-4-turbo',
                 'togetherai/meta-llama/Meta-Llama-3-70B-Instruct-Turbo',
                                      'togetherai/google/gemma-2-9b-it',
                 ],
                      prefix="",
                      skip=0,
    MyCascade = FrugalGPT.LLMCascade(
          score_noise_injection=False,
  db_path="db/SCIQ.sqlite",
  ),

    cascade_depth=3,
score_test_size=0.55,

                      ):

  for idx,budget in tqdm(enumerate(budget_list)):
    # train the model
    user_budget = budget
    try:
      MyCascade.load(loadpath=f"strategy/{name}/",budget=user_budget)
      print("Already trained. Skipped.")
      continue
    except:
      print("cannot find, start new training")
    if(idx<skip):
      continue
    if(idx==0):
        result = MyCascade.train(train_data,budget=user_budget,
                                 service_names=service_names,
                                 no_scorer_train=False,
                                 prefix=prefix,
                                 cascade_depth=cascade_depth,
                                 score_test_size=score_test_size,
                                 )
    else:
      result = MyCascade.train(train_data,budget=user_budget,
                               service_names=service_names,
                               no_scorer_train=True,
                               prefix=prefix,
                               cascade_depth=cascade_depth,
                               score_test_size=score_test_size,
                               )
    MyCascade.save(savepath=f"strategy/{name}/")
  return

In [23]:
start_budget = 0.000074
end_budget = 0.016
num_eval = 20

name = f'{dataname}_Model202411'
budget_list = numpy.linspace(start_budget, end_budget, num_eval)
budget_list[0] = 0.00007
#budget_list = budget_list[::-1]

# load data
dev = FrugalGPT.loadcsvdata(f"data/{dataname}/train.csv")
train_data = FrugalGPT.formatdata(dev,prefix)
MyCascade= FrugalGPT.LLMCascade(
          score_noise_injection=False,
  db_path=f"db/{dataname}.sqlite",
  batch_build=True,
  )
#MyCascade.load(loadpath=f"strategy/{name}/",budget=0.00017)

In [22]:
service_names_train = [
    'togetherai/meta-llama/Meta-Llama-3-70B-Instruct-Turbo',

    'google/gemini-1.5-flash',
    'google/gemini-1.5-flash-8b',

    #'openaichat/gpt-4o-mini',
    'openaichat/gpt-4-turbo',

    'anthropic/claude-3-5-sonnet-20240620',
                 ]

compute_tradeoffs(train_data=train_data,
                  budget_list=budget_list,
                  name=name,
                  service_names=service_names_train,
                  prefix=prefix,
                  skip=0, # you can manually skip the first few budgets if they have already been trained.
                  MyCascade=MyCascade,
                  cascade_depth=3,
                  score_test_size=0.55,
                  )

1it [00:02,  2.01s/it]

Already trained. Skipped.


2it [00:04,  2.03s/it]

Already trained. Skipped.


3it [00:06,  2.04s/it]

...

18it [00:36,  2.07s/it]

Already trained. Skipped.


19it [00:38,  2.07s/it]

Already trained. Skipped.
cannot find, start new training
first train


20it [06:23, 19.16s/it] 


In [ ]:
import shutil

# Specify the folder to zip
folder_to_zip = f'strategy/{name}'
output_zip_file = f'{name}.zip'

# Create the zip file
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_zip)
print(f"Folder '{folder_to_zip}' zipped as '{output_zip_file}'.")

In [ ]:
files.download(output_zip_file)

## Step 3: Evaluate and save the performance

In [ ]:
def generate_dataframe_from_cascade(MyCascade,budget_list, train_data, test_data, genparams,name):
    # Initialize an empty list to store the rows for the DataFrame
    data = []

    # Iterate through the budget list
    for budget in tqdm(budget_list):
        # Load the strategy for the given budget
        MyCascade.load(loadpath=f"strategy/{name}/", budget=budget)

        # Get the completion batch for train data
        train_result = MyCascade.get_completion_batch(queries=train_data, genparams=genparams)

        # Compute the ACC and cost for train data
        train_acc_cost = FrugalGPT.compute_score(train_result)


        # Get the completion batch for test data
        test_result = MyCascade.get_completion_batch(queries=test_data, genparams=genparams)

        # Compute the ACC and cost for test data
        test_acc_cost = FrugalGPT.compute_score(test_result)

        # Create a row with the schema
        row = {
            "Test_acc": test_acc_cost['em'],
            "Test_cost": test_acc_cost['cost'],
            "Test_size": len(test_data),
            "Train_acc": train_acc_cost['em'],
            "Train_cost": train_acc_cost['cost'],
            "Train_size": len(train_data),
            "Budget": budget,
            "Method": "FrugalGPT",
            "Provider": "FrugalGPT",
            "Marker": 1,  # Marker is always 1 for this function
        }

        # Append the row to the data list
        data.append(row)
        display(row)

    # Create the DataFrame from the data list
    df = pd.DataFrame(data)

    return df

In [24]:
MyCascade_eval = FrugalGPT.LLMCascade()
MyCascade_eval.prefix = prefix
frugalgpt_df = generate_dataframe_from_cascade(MyCascade_eval,
                                               budget_list, train_data, test_data, genparams,
                                               name)
display(frugalgpt_df)
frugalgpt_df.to_csv(f"summary_{dataname}_e8_frugalgpt_2024.csv")

  0%|          | 0/20 [00:00<?, ?it/s]

{'Test_acc': 0.24154347281383112,
 'Test_cost': 6.755906101227764e-05,
 'Test_size': 3991,
 'Train_acc': 0.2593335003758456,
 'Train_cost': 6.851799799548985e-05,
 'Train_size': 3991,
 'Budget': 7e-05,
 'Method': 'FrugalGPT',
 'Provider': 'FrugalGPT',
 'Marker': 1}

  5%|▌         | 1/20 [01:24<26:48, 84.65s/it]

{'Test_acc': 0.3693309947381609,
 'Test_cost': 0.0007175995684039088,
 'Test_size': 3991,
 'Train_acc': 0.38185918316211476,
 'Train_cost': 0.0007050719055374592,
 'Train_size': 3991,
 'Budget': 0.0009122105263157893,
 'Method': 'FrugalGPT',
 'Provider': 'FrugalGPT',
 'Marker': 1}

 10%|█         | 2/20 [04:13<40:14, 134.12s/it]

...

{'Test_acc': 0.3838636933099474,
 'Test_cost': 0.003916803316211476,
 'Test_size': 3991,
 'Train_acc': 0.40090202956652465,
 'Train_cost': 0.0039059640027562016,
 'Train_size': 3991,
 'Budget': 0.015161789473684208,
 'Method': 'FrugalGPT',
 'Provider': 'FrugalGPT',
 'Marker': 1}

 95%|█████████▌| 19/20 [49:18<02:42, 162.91s/it]

{'Test_acc': 0.3838636933099474,
 'Test_cost': 0.003916803316211476,
 'Test_size': 3991,
 'Train_acc': 0.40090202956652465,
 'Train_cost': 0.0039059640027562016,
 'Train_size': 3991,
 'Budget': 0.016,
 'Method': 'FrugalGPT',
 'Provider': 'FrugalGPT',
 'Marker': 1}

100%|██████████| 20/20 [52:01<00:00, 156.09s/it]


,Test_acc,Test_cost,Test_size,Train_acc,Train_cost,Train_size,Budget,Method,Provider,Marker
0,0.241543,0.000068,3991,0.259334,0.000069,3991,0.000070,FrugalGPT,FrugalGPT,1
1,0.369331,0.000718,3991,0.381859,0.000705,3991,0.000912,FrugalGPT,FrugalGPT,1
2,0.381609,0.001700,3991,0.389877,0.001735,3991,0.001750,FrugalGPT,FrugalGPT,1
3,0.385618,0.002445,3991,0.393886,0.002420,3991,0.002589,FrugalGPT,FrugalGPT,1
4,0.375595,0.003355,3991,0.395640,0.003355,3991,0.003427,FrugalGPT,FrugalGPT,1
5,0.383864,0.003917,3991,0.400902,0.003906,3991,0.004265,FrugalGPT,FrugalGPT,1
6,0.383864,0.003917,3991,0.400902,0.003906,3991,0.005103,FrugalGPT,FrugalGPT,1
7,0.383864,0.003917,3991,0.400902,0.003906,3991,0.005941,FrugalGPT,FrugalGPT,1
8,0.383864,0.003917,3991,0.400902,0.003906,3991,0.006780,FrugalGPT,FrugalGPT,1
9,0.383864,0.003917,3991,0.400902,0.003906,3991,0.007618,FrugalGPT,FrugalGPT,1


Now let us save the results to local disk.

In [ ]:
from google.colab import files
import copy
individualmodel_df2 = copy.copy(individualmodel_df)
#individualmodel_df2['Test_cost'] = individualmodel_df2['Test_cost'] * individualmodel_df2['Test_size']
full_pd = pd.concat([frugalgpt_df,individualmodel_df2,])
full_pd.to_csv(f"summary_{dataname}_e2_full_2024.csv")
files.download(f'summary_{dataname}_e2_full_2024.csv')
display(full_pd)